### This is my first (public) notebook of playing with pyspark

The goal of this notebook is to demonstrare a simple use case of:
1. using pyspark (python wrapper for spark) and access existing spark cluster
2. loading data into spark
3. accessing the data using the rdd-api, dataframe-api, and dataset-api
4. simple data exploration

I'll be using a sample of the gutenberg project - https://web.eecs.umich.edu/~lahiri/gutenberg_dataset.html

#### The settings I'm using here:
- spark: version 2.0.0
- python: 3.5

In [ ]:
#import urllib3 #ability to access data by url
import pyspark #python wrapper for spark
sc.stop()
sc = pyspark.SparkContext(appName="spark-project")

In [ ]:
# Here we demonstrare a usage of the RDD API
rdd = sc.textFile("s3n://ak-public-sandbox/datasets/gutenberg_dataset/1/8/*/*/*")
rdd.count()
rdd.take(1)

In [ ]:
#Let's see some fitering capabilities
rdd.filter(lambda line: "mankind" in line).count()

In [ ]:
#the good old word count
counts = rdd.flatMap(lambda line: line.split(" ")).map(lambda word: (word,1)).reduceByKey(lambda a,b: a + b)

In [ ]:
# some word-level stats
from pprint import pprint
stop_words = counts.takeOrdered(10,lambda a:-a[1])
rare_words = counts.takeOrdered(10,lambda a:a[1])
print("Stop words:")
pprint(stop_words)
print("Rare words:")
pprint(rare_words)
print("total number of uniqu words %s" % counts.count())

In [ ]:
rdd.filter(lambda line: "mankind" in line).saveAsTextFile("s3n://spark-course-data2/result/lesson_2-2")

In [ ]:
mankindLines = sc.textFile("s3n://spark-course-data2/result/lesson_2-2")
mankindLines.count()

In [ ]:
rdd = sc.wholeTextFiles("s3n://spark-course-data2/gutenberg_dataset/1/8/*/*/*")
rdd.count()
rdd.take(1)

In [ ]:
import boto3
session = boto3.Session() # leave out the profile_name argument if you haven't defined profiles
s3 = boto3.resource('s3')
bucket = s3.Bucket('spark-course-data2')
objs = bucket.objects.filter(Prefix='gutenberg_dataset/1')
for obj in objs:  
  print(obj.key)

In [ ]:
obj = s3.Object('spark-course-data2', 'gutenberg_dataset/1/9/9/1997/1997.txt')
content = obj.get()["Body"].read().decode("utf-8")
print(content[:100])

In [ ]:
http = urllib3.PoolManager()
r = http.request('GET', 'https://www.kaggle.com/c/titanic/download/train.csv')
r.status
#response = urllib3.urlopen('')
#html = response.read()